In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

### Importujemy potrzebne biblioteki

### autorzy: Adrian Bloch i Witold Jagiełło

# Rozpoznawanie zwięrząt na podstawie CIFAR10 -CNN

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### jeśli dostępny jest procesor graficzny to go wykorzystaj inaczej użyj procesora

In [22]:
num_epochs = 4
batch_size = 4
learning_rate = 0.001

### parametry

In [23]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

### transformacja obrazów z zakresu [0,1] na tensory o zakresie [-1,1]

In [24]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### dane testowe oraz treningowe załadowane dzięki torchvision

In [25]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Możliwe zwierzęta

In [26]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Tworzymy klasę ConvNet, która dziedziczy klasę z nn.Module czyli superklasą wszystkich sieci neuronowych kiedy programujemy wykorzystując pytorch

In [27]:
def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### przekazywanie dalej

In [28]:
model = ConvNet().to(device)

### Tworzymy obiekt

In [29]:
criterion = nn.CrossEntropyLoss()

### definiujemy funkcję, która jest w stanie wskazać nam straty

In [30]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### implementujemy przypadkową zmianę koloru zgodną ze zmianą pędu

In [31]:
n_total_steps = len(train_loader)

### pętla do treningu

In [32]:
for epoch in range(num_epochs):

    for i, (images, labels) in enumerate(train_loader):

        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 2500 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {loss.item():.4f}')
print('Koniec treningu')

Epoch [1/4], Step [2500/12500], Loss: 2.3018
Epoch [1/4], Step [5000/12500], Loss: 2.3057
Epoch [1/4], Step [7500/12500], Loss: 2.2611
Epoch [1/4], Step [10000/12500], Loss: 2.2877
Epoch [1/4], Step [12500/12500], Loss: 2.0893
Epoch [2/4], Step [2500/12500], Loss: 1.6483
Epoch [2/4], Step [5000/12500], Loss: 1.9071
Epoch [2/4], Step [7500/12500], Loss: 1.4281
Epoch [2/4], Step [10000/12500], Loss: 1.8755
Epoch [2/4], Step [12500/12500], Loss: 1.6323
Epoch [3/4], Step [2500/12500], Loss: 1.7031
Epoch [3/4], Step [5000/12500], Loss: 1.3831
Epoch [3/4], Step [7500/12500], Loss: 1.5419
Epoch [3/4], Step [10000/12500], Loss: 1.0076
Epoch [3/4], Step [12500/12500], Loss: 1.5798
Epoch [4/4], Step [2500/12500], Loss: 1.9777
Epoch [4/4], Step [5000/12500], Loss: 1.8169
Epoch [4/4], Step [7500/12500], Loss: 1.4017
Epoch [4/4], Step [10000/12500], Loss: 2.7290
Epoch [4/4], Step [12500/12500], Loss: 1.3084
Koniec treningu


### zapętlamy w pętli dzięki pętli treningowej i wyrzucamy obraz na GPU

In [33]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.00 * n_correct / n_samples
    print(f'Dokładność sieci: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Dokładność {classes[i]}: {acc} %')

Dokładność sieci: 46.71 %
Dokładność plane: 51.38 %
Dokładność car: 68.72 %
Dokładność bird: 30.86 %
Dokładność cat: 31.14 %
Dokładność deer: 29.78 %
Dokładność dog: 48.36 %
Dokładność frog: 48.18 %
Dokładność horse: 59.84 %
Dokładność ship: 57.6 %
Dokładność truck: 41.24 %


### pokazujemy dokładność sieci i każdej z klas